In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances

# Load metadata
meta = pd.read_csv("../data/bicikelj_metadata.csv")
coords = np.deg2rad(meta[['latitude', 'longitude']].values)

# Compute distance matrix (in km)
dists = haversine_distances(coords) * 6371

# Build adjacency: connect to 4 nearest neighbors
adj = np.zeros_like(dists)
for i in range(dists.shape[0]):
    nn = np.argsort(dists[i])[1:5]  # skip self, get 4 nearest
    adj[i, nn] = 1


In [5]:
adj

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# --- Define basic Graph Convolution layer ---
class GraphConv(nn.Module):
    def __init__(self, in_channels, out_channels, adj):
        super().__init__()
        self.adj = torch.FloatTensor(adj)
        self.weight = nn.Parameter(torch.randn(in_channels, out_channels) * 0.1)

    def forward(self, x):
        # x: [batch, nodes, in_channels]
        out = torch.einsum('ij,bjk->bik', self.adj, x)  # message passing
        out = torch.einsum('bik,kl->bil', out, self.weight)
        return out

# --- STGCN Block ---
class STGCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, adj):
        super().__init__()
        self.gconv = GraphConv(in_channels, out_channels, adj)
        self.tconv = nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=(3,1), padding=(1,0)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        # x: [batch, seq_len, nodes]
        batch, seq_len, nodes = x.shape
        # Temporal conv
        x = x.unsqueeze(1)  # [batch, 1, seq_len, nodes]
        x = self.tconv(x)
        x = x.squeeze(1)    # [batch, seq_len, nodes]
        # Graph conv at each time step
        out = []
        for t in range(x.shape[1]):
            xt = x[:, t, :] # [batch, nodes]
            xt = xt.unsqueeze(-1) # [batch, nodes, 1]
            out.append(self.gconv(xt).squeeze(-1)) # [batch, nodes]
        x = torch.stack(out, dim=1) # [batch, seq_len, nodes]
        return self.relu(x)

# --- Full Model ---
class STGCN(nn.Module):
    def __init__(self, adj, num_nodes, input_len=48, output_len=4):
        super().__init__()
        self.block1 = STGCNBlock(1, 16, adj)
        self.block2 = STGCNBlock(16, 32, adj)
        self.fc = nn.Linear(input_len*32, output_len)

    def forward(self, x):
        # x: [batch, seq_len, nodes]
        x = x.permute(0, 2, 1).unsqueeze(-1) # [batch, nodes, seq_len, 1]
        x = x.squeeze(-1).permute(0, 2, 1)   # [batch, seq_len, nodes]
        x = self.block1(x)
        x = self.block2(x)
        # Flatten time/features for each node
        x = x.permute(0, 2, 1) # [batch, nodes, seq_len*features]
        x = x.reshape(x.shape[0], x.shape[1], -1)
        # Predict next 4 hours for each node
        out = self.fc(x) # [batch, nodes, output_len]
        # Return to [batch, output_len, nodes]
        return out.permute(0, 2, 1)

# --- Training Loop Example ---
def train_stgcn(X_train, y_train, adj, num_epochs=10, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_nodes = X_train.shape[2]
    model = STGCN(adj, num_nodes, input_len=X_train.shape[1], output_len=y_train.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    X = torch.FloatTensor(X_train).to(device)
    y = torch.FloatTensor(y_train).to(device)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(X)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs} Loss: {loss.item():.4f}")
    return model

# --- Example Data Preparation (dummy, replace with your own) ---
# Suppose N_samples = 1000, input_len = 48, output_len = 4, num_nodes = 10
N_samples, input_len, output_len, num_nodes = 1000, 48, 4, 10
X_train = np.random.randn(N_samples, input_len, num_nodes)
y_train = np.random.randn(N_samples, output_len, num_nodes)
adj = np.eye(num_nodes) # Replace with your adjacency matrix

# --- Train the model ---
model = train_stgcn(X_train, y_train, adj, num_epochs=20)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_bmm)